# Network and relations making

In [9]:
import subprocess
import os
import time
from yml_create_functions import make_yml, make_network_agents_yml

In [10]:
name_simulation = 'simulation_tutorial'

## Network Agent Parameters

The schema can be changed in schema/schema.py. Here, you can change the parameters of the default schema

In [12]:
network_agents_parameters = {
    "default": {
        "id_message": "NaN",
        "has_tv": "false",
        "cause": -1,
        "method": "NaN",

        "type": "dumb",
        "response": "NaN",
        "stance": "agree",
        "repost": "NaN",
        "parent_id": "NaN"

    },

    "DumbViewer": [
        {"weight": 2},
        {"weight": 2, "has_tv": "true"}
    ],
    "HerdViewer": [
        {"weight": 2, "type": "herd", "stance": "against"},
        {"weight": 2, "type": "herd", "has_tv": "true"}
    ],
    "WiseViewer": [
        {"weight": 1, "type": "wise", "stance": "against"},
        {"weight": 1, "type": "wise", "has_tv": "true", "stance": "against"}
    ],

}

## Other Parameters

In [13]:
parameters = {
    "default_state": "{}",
    "load_module": "schema", # path from yml to schema
    "environment_agents": "[]",
    "environment_class": "schema.NewsEnvironmentAgent", # path from yml to environment class
    "environment_params": {
        "prob_neighbor_spread": 0.02,  # 006
        "prob_tv_spread": 0.006,  # 001
        "prob_neighbor_cure": 0.006,  # 001
        "prob_backsliding": 0.01,  # 003
        "prob_dead": 0.006,  # 001,
        "prob_repost": 0.8,
        "hostility_weight": 1,  # Unused yet
        "mean_time_connection": 50,
        "var_time_connection": 30
    },
    "interval": 1,
    "max_time": 100,
    "name": name_simulation,
    "network_params": {
        "generator": "barabasi_albert_graph",
        "n": 100,
        "m": 5
    },
    "num_trials": 1
}

## Simulation execution

In [14]:
parameters_i = parameters.copy()
network_agents_parameters_i = network_agents_parameters.copy()
data = make_yml(parameters_i)
data += make_network_agents_yml(network_agents_parameters_i)

yml_path = os.path.join('schema', '{name_simulation}.yml') # YML path
with open(yml_path, 'w') as file:
    file.write(data)

command = "soil"
start = time.time()

output = subprocess.check_output([command, yml_path])
end = time.time()
seconds_simulation = str(end-start)
print("SIMULATION'S SECONDS:", seconds_simulation)

SIMULATION'S SECONDS: 5.391209840774536


# Conversation Making

In [15]:
from get_data.get_data import get_pivoted_data
from post.templates import POST_TEMPLATE, REPLY_TEMPLATE
from anytree import Node, RenderTree, LevelOrderIter
from post.post import Post
from IA.ia import send_prompt

## Output sqlite path

In [16]:
analysis_path = os.path.join('soil_output', name_simulation)
sql_table_path = f'{name_simulation}_trial_0.sqlite'

## Get Data

In [17]:
data = get_pivoted_data(analysis_path, sql_table_path)

## Define News

In [18]:
NEWS = 'Chilean Government Announces Ambitious Plan for Sustainable Energy Transition'
SHOW_TREE = True

## Create the Network's Tree

In [19]:
root = Post(0, message=NEWS, post_template=POST_TEMPLATE,
             reply_template=REPLY_TEMPLATE)
lista_nodos = [root] + [Post("", message='Empty') for _ in range(len(data))]

In [20]:
for i in range(len(data)):
    aux = data[i]
    owner = aux[0]
    cause = aux[3]
    method = aux[4]
    response = aux[5]
    stance = aux[6]
    repost = aux[7]
    if repost == 'true':
        repost = True
    elif repost == 'false':
        repost = False
    else:
        print(data[i])
        print(repost)
        raise ValueError("Error repost format")
    
    name = aux[8]
    parent = lista_nodos[int(aux[9])]

    message = ''

    lista_nodos[i+1] = Post(name, parent=parent, owner=owner,
                             message=message, stance=stance,
                             repost=repost, cause=cause,
                             method=method, response=response,
                             post_template=POST_TEMPLATE,
                             reply_template=REPLY_TEMPLATE,
                             news=NEWS)

## Associate a Written Post to each Message

In [21]:
from parameters import API_KEY
import openai
openai.api_key = API_KEY

In [22]:
list_messages = []
for node_i in LevelOrderIter(root):
    if node_i.repost or node_i == root:
        continue
    prompt = node_i.get_prompt(language='english', 
                               min_caract=100, max_caract=180)
    answer = send_prompt(prompt, temp=1.5)
    node_i.set_message(answer)
    list_messages.append((node_i, prompt))

## Plot the Network in a Tree

In [23]:
if SHOW_TREE:
    for pre, _, node in RenderTree(root):
        if node.repost:
            message = "repost"
        else:
            message = "'"+node.message+"'"
        
        if node.name == 0:
            print(f"{pre}NEWS: {message}")
        else:
            print(f"{pre}{node.owner} {message}")

NEWS: 'Chilean Government Announces Ambitious Plan for Sustainable Energy Transition'
├── 32 'That sounds awesome! The Chilean government is taking a big step towards a sustainable future! This plan will surely make a positive impact ♻️ Here's to cleaner energy and a healthier planet 🌍✨ #SustainabilityMatters.'
│   └── 4 'Agreed! It's great to see the Chilean government prioritizing sustainability and taking solid measures to ensure a greener future. By transitioning to sustainable energy sources, we'll combat climate change and protect our beautiful planet. Can't wait to witness the positive impact this plan will create! 🌍💚 #ClimateAction'
├── 40 'wow, the Chilean government really thinks this sustainable energy plan is the way to go. 🙄 but is it even practical? I mean, are they just talking a big game or do they have any real strategies in place? 🤷‍♀️ can't just rely on renewable energy without thinking about the cost or stability. 😬 time will tell, I guess. Hope they know what they'